In [11]:
import pandas as pd

# Load the data from the provided CSV files
customers_df = pd.read_csv('Customers.csv', encoding='utf-8')
transactions_df = pd.read_csv('Transactions.csv', encoding='utf-8')

# Display the first few rows of each dataframe to understand their structure
customers_df.head(), transactions_df.head()

(  CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera  South America  2024-03-07
 3      C0004  Kathleen Rodriguez  South America  2022-10-09
 4      C0005         Laura Weber           Asia  2022-08-15,
   TransactionID CustomerID ProductID      TransactionDate  Quantity   
 0        T00001      C0199      P067  2024-08-25 12:38:23         1  \
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  2024-04-25 07:38:55         1   
 3        T00272      C0087      P067  2024-03-26 22:55:37         2   
 4        T00363      C0070      P067  2024-03-21 15:10:10         3   
 
    TotalValue   Price  
 0      300.68  300.68  
 1      300.68  300.68  
 2      300.68  300.68  
 3      601.36  300.68  
 4      902.04  300.68  )

In [12]:
# Merge the customers and transactions dataframes on CustomerID
merged_df = pd.merge(customers_df, transactions_df, on='CustomerID', how='inner')

# Display the first few rows of the merged dataframe
merged_df.head()

,CustomerID,CustomerName,Region,SignupDate,TransactionID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,C0001,Lawrence Carroll,South America,2022-07-10,T00015,P054,2024-01-19 03:12:55,2,114.60,57.30
1,C0001,Lawrence Carroll,South America,2022-07-10,T00932,P022,2024-09-17 09:01:18,3,412.62,137.54
2,C0001,Lawrence Carroll,South America,2022-07-10,T00085,P096,2024-04-08 00:01:00,2,614.94,307.47
3,C0001,Lawrence Carroll,South America,2022-07-10,T00445,P083,2024-05-07 03:11:44,2,911.44,455.72
4,C0001,Lawrence Carroll,South America,2022-07-10,T00436,P029,2024-11-02 17:04:16,3,1300.92,433.64


In [13]:
# Calculate total transactions, total quantity, and total spend for each customer
customer_features = merged_df.groupby('CustomerID').agg(
    TotalTransactions=('TransactionID', 'count'),
    TotalQuantity=('Quantity', 'sum'),
    TotalSpend=('TotalValue', 'sum')
).reset_index()

# Display the first few rows of the customer features dataframe
customer_features.head()

,CustomerID,TotalTransactions,TotalQuantity,TotalSpend
0,C0001,5,12,3354.52
1,C0002,4,10,1862.74
2,C0003,4,14,2725.38
3,C0004,8,23,5354.88
4,C0005,3,7,2034.24


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import numpy as np

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features[['TotalTransactions', 'TotalQuantity', 'TotalSpend']])

# Initialize variables to store the best number of clusters and the corresponding DB Index
best_num_clusters = None
best_db_index = np.inf

# Try different numbers of clusters
for n_clusters in range(2, 11):
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_features)
    
    # Calculate the Davies-Bouldin Index
    db_index = davies_bouldin_score(scaled_features, cluster_labels)
    
    # Update the best number of clusters if the current DB Index is lower
    if db_index < best_db_index:
        best_db_index = db_index
        best_num_clusters = n_clusters

best_num_clusters, best_db_index

(2, 0.7233652695141874)

In [16]:
import plotly.express as px

# Apply K-Means clustering with the best number of clusters
kmeans = KMeans(n_clusters=best_num_clusters, random_state=42)
customer_features['Cluster'] = kmeans.fit_predict(scaled_features)

# Visualize the clusters
fig = px.scatter(
    customer_features, x='TotalQuantity', y='TotalSpend', color='Cluster',
    title='Customer Segmentation',
    labels={'TotalQuantity': 'Total Quantity', 'TotalSpend': 'Total Spend'},
    color_continuous_scale=px.colors.qualitative.Plotly
)
fig.show()